In [17]:
import os
import re, string
import pandas as pd
import numpy as np
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from gensim.models import Phrases
from gensim.corpora.dictionary import Dictionary
from numpy import array
from gensim.models.ldamulticore import LdaMulticore
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
from bs4 import BeautifulSoup

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lalsk\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
articles = []
files_names = []
for dirname, _, filenames in os.walk('C:/Users/lalsk/Desktop/d/статьи/conv'):
    for filename in filenames:
        #print(filename)
        my_file = os.path.join(dirname, filename)
        file = open(my_file,'r',encoding = "utf-8")
        temp = file.read()
        files_names.append(filename)
        articles.append(temp)

In [20]:
files_res_names = []
udc = []
for dirname, _, filenames in os.walk('C:/Users/lalsk/Desktop/d/статьи/res'):
    for filename in filenames:
        if(filename != "desktop.ini"):
            my_file = os.path.join(dirname, filename)
            with open(my_file, "r") as f:
                contents = f.read()
                soup = BeautifulSoup(contents, 'lxml')
            #print(filename)
            u1 = soup.find(text='УДК:')
            if (u1 != None):
                u2 = u1.parent.parent
                udc.append(u2.text.strip())
            else:
                udc.append(0)
            files_res_names.append(filename)  

In [21]:
udc1 = udc.copy()
articles1 = articles.copy()
files_names1 = files_names.copy()
files_res_names1 = files_res_names.copy()

In [22]:
for i in range(len(udc)):
    if udc[i] != 0:
        udc[i] = udc[i].replace('\n','')
        udc[i] = udc[i].replace('УДК:','')
        udc[i] = udc[i].replace('\t','')
        udc[i] = re.sub(r'[+,;,:,/]', ',', udc[i])

In [23]:
for i in range(len(udc)):
    if udc[i] != 0:
        if ',' in udc[i]:
            temp = udc[i].split(',')
            udc[i] = temp[0]

In [24]:
U = ['51-3','51-7','51-8','510','511','512','514','515.1','517','519.1','519.2','519.6','519.7','519.8']
for i in range(len(udc)):
    if udc[i] != 0:
        for j in range(len(U)):
            if U[j] in udc[i]:
                udc[i] = U[j]

In [25]:
for i in range(len(files_names)):
    files_names[i] = files_names[i].replace('.pdf convert.txt','')
for i in range(len(files_res_names)):
    files_res_names[i] = files_res_names[i].replace('-resume.html','')

In [9]:
#for i in range(len(articles)):
#    articles[i] = articles[i].replace('\n',' ')
#    articles[i] = re.sub(r'[.,"\'-?:!;]', '', articles[i])
#    articles[i] = re.sub(r'[a-zA-Z]', '', articles[i])
#    articles[i] = re.sub(r'[λ,τ,χ,σ,ε,×,∈,−,ω,∩,⊆,≤,α,ξ,∆,η,£,•,—,✷,ω,∀,∃,ℵ,¬,β,π,σ,∼,§,❅,γ,θ,¯,→,∗,№,≺,ψ,≡,∨,ϕ,∞,δ,⊥,❁,⊕,◦,⊗,µ,∪,·,∂,∅,ρ]', '', articles[i])
#    articles[i] = re.sub(r'\s+[а-яА-Я]\s+', ' ', articles[i])
#    articles[i] = re.sub(r'\^[а-яА-Я]\s+', ' ', articles[i])
#    articles[i] = re.sub(r'\s+[а-яА-Я]\^', ' ', articles[i]) 

In [26]:
df1 = pd.DataFrame()
df1['name'] = files_names
df1['article'] = articles
df2 = pd.DataFrame()
df2['name'] = files_res_names
df2['udc'] = udc
df_articles = pd.merge(df1,df2,on='name')
df_articles

,name,article,udc
0,de-925,﻿\nEvaluation Only. Created with Aspose.Words....,517


In [27]:
df_articles=df_articles[df_articles['udc'] != 0]
df_articles

,name,article,udc
0,de-925,﻿\nEvaluation Only. Created with Aspose.Words....,517


In [12]:
df_articles.to_csv('C:/Users/lalsk/Desktop/d/статьи/articles.csv')

In [28]:
stop = ['это', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', 'км','наш','который', 'теорема', 'лемма','Общероссийский',
           'математический','портал']
stop_tags = stopwords.words('russian') + stop + stopwords.words('english')
regex = re.compile('[%s]' % re.escape(string.punctuation)) 
stemmer = SnowballStemmer("russian")

In [29]:
def preprocessing(text):
    text = text.replace('\n',' ')
    text = re.sub(r'[.,"\'-?:!;]', '', text)
    text = re.sub(r'[a-zA-Z]', '', text)
    text = re.sub(r'[λ,τ,χ,σ,ε,×,∈,−,ω,∩,⊆,≤,α,ξ,∆,η,£,•,—,✷,ω,∀,∃,ℵ,¬,β,π,σ,∼,§,❅,γ,θ,¯,→,∗,№,≺,ψ,≡,∨,ϕ,∞,δ,⊥,❁,⊕,◦,⊗,µ,∪,·,∂,∅,ρ]', '', text)
    text = re.sub(r'\s+[а-яА-Я]\s+', ' ', text)
    text = re.sub(r'\^[а-яА-Я]\s+', ' ', text)
    text = re.sub(r'\s+[а-яА-Я]\^', ' ', text)
    text = regex.sub('', text) # удаляем пунктуацию
    text = re.sub(r'^b\s+', '', text)
    text = [token for token in text.split(' ') if token not in stop_tags] # Удаляем стоп слова
    text = [stemmer.stem(token) for token in text] # Выполняем стэмминг        
    text = [token for token in text if token] # Удаляем пустые токены
    return ' '.join(text)

In [30]:
prep_articles = []
for i in range(len(df_articles['article'])):
    prep_articles.append(preprocessing(df_articles['article'].tolist()[i].lower()))

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(prep_articles, df_articles['udc'].tolist(), test_size=0.14, random_state=0)

обучающая выборка

In [17]:
text_clean_train= []
for i in range(len(X_train)):
    temp = []
    for j in list(X_train[i].lower().split(' ')):
                temp.append(j.split())
    text_clean_train.append(temp)

In [18]:
bigrams_train = []
trigrams_train= []
for i in range(len(text_clean_train)):
    bigram = Phrases(text_clean_train[i]) # Создаем биграммы на основе корпуса
    trigram = Phrases(bigram[text_clean_train[i]])# Создаем триграммы на основе корпуса
    bigrams_train.append(bigram)
    trigrams_train.append(trigram)

In [19]:
for j in range(len(text_clean_train)):
    for i in range(len(text_clean_train[j])):
        for token in bigrams_train[j][text_clean_train[j][i]]:
            if '_' in token:
                # Токен это биграмма, добавим в документ.
                text_clean_train[j][i].append(token)
        for token in trigrams_train[j][text_clean_train[j][i]]:
            if '_' in token:
                text_clean_train[j][i].append(token)

словарь

In [20]:
clean= []
for i in range(len(X_train)):
    for j in list(X_train[i].lower().split(' ')):
                clean.append(j.split())

In [21]:
bigram_ = Phrases(clean) # Создаем биграммы на основе корпуса
trigram_ = Phrases(bigram[clean])# Создаем триграммы на основе корпуса

In [22]:
for i in range(len(clean)):
    for token in bigram[clean[i]]:
        if '_' in token:
            # Токен это биграмма, добавим в документ.
            clean[i].append(token)
    for token in trigram[clean[i]]:
        if '_' in token:
            clean[i].append(token)

In [23]:
dictionary = Dictionary(clean)
dictionary.filter_extremes(no_below=10, no_above=0.1)

_______

In [24]:
for i in range(len(text_clean_train)):
    for j in range(len(text_clean_train[i])):
        text_clean_train[i][j].append(' 0')

In [25]:
list_topics_train = []
count = 0
for i in range(len(X_train)):
    count+=1
    print(count)
    corpus = [dictionary.doc2bow(doc) for doc in text_clean_train[i]]
    print('Количество уникальных токенов: %d' % len(dictionary))
    print('Количество документов: %d' % len(corpus))
    model=LdaMulticore(corpus=corpus,id2word=dictionary, num_topics=14)
    temp_list = []
    if (model != 0):
        for j in range(4):
            temp_list.append(model.show_topic(j,topn=14))
    else: 
        temp_list.append('0')
    list_topics_train.append(temp_list)

1
Количество уникальных токенов: 13284
Количество документов: 977
2
Количество уникальных токенов: 13284
Количество документов: 866
3
Количество уникальных токенов: 13284
Количество документов: 539
4
Количество уникальных токенов: 13284
Количество документов: 481
5
Количество уникальных токенов: 13284
Количество документов: 1350
6
Количество уникальных токенов: 13284
Количество документов: 1116
7
Количество уникальных токенов: 13284
Количество документов: 1757
8
Количество уникальных токенов: 13284
Количество документов: 744
9
Количество уникальных токенов: 13284
Количество документов: 983
10
Количество уникальных токенов: 13284
Количество документов: 1516
11
Количество уникальных токенов: 13284
Количество документов: 334
12
Количество уникальных токенов: 13284
Количество документов: 1434
13
Количество уникальных токенов: 13284
Количество документов: 976
14
Количество уникальных токенов: 13284
Количество документов: 1022
15
Количество уникальных токенов: 13284
Количество документов: 86

123
Количество уникальных токенов: 13284
Количество документов: 890
124
Количество уникальных токенов: 13284
Количество документов: 938
125
Количество уникальных токенов: 13284
Количество документов: 694
126
Количество уникальных токенов: 13284
Количество документов: 683
127
Количество уникальных токенов: 13284
Количество документов: 931
128
Количество уникальных токенов: 13284
Количество документов: 1804
129
Количество уникальных токенов: 13284
Количество документов: 1216
130
Количество уникальных токенов: 13284
Количество документов: 1032
131
Количество уникальных токенов: 13284
Количество документов: 360
132
Количество уникальных токенов: 13284
Количество документов: 702
133
Количество уникальных токенов: 13284
Количество документов: 1303
134
Количество уникальных токенов: 13284
Количество документов: 587
135
Количество уникальных токенов: 13284
Количество документов: 1465
136
Количество уникальных токенов: 13284
Количество документов: 426
137
Количество уникальных токенов: 13284
Ко

243
Количество уникальных токенов: 13284
Количество документов: 968
244
Количество уникальных токенов: 13284
Количество документов: 2879
245
Количество уникальных токенов: 13284
Количество документов: 1231
246
Количество уникальных токенов: 13284
Количество документов: 738
247
Количество уникальных токенов: 13284
Количество документов: 1077
248
Количество уникальных токенов: 13284
Количество документов: 473
249
Количество уникальных токенов: 13284
Количество документов: 1463
250
Количество уникальных токенов: 13284
Количество документов: 372
251
Количество уникальных токенов: 13284
Количество документов: 1187
252
Количество уникальных токенов: 13284
Количество документов: 1042
253
Количество уникальных токенов: 13284
Количество документов: 854
254
Количество уникальных токенов: 13284
Количество документов: 1018
255
Количество уникальных токенов: 13284
Количество документов: 233
256
Количество уникальных токенов: 13284
Количество документов: 756
257
Количество уникальных токенов: 13284


363
Количество уникальных токенов: 13284
Количество документов: 440
364
Количество уникальных токенов: 13284
Количество документов: 777
365
Количество уникальных токенов: 13284
Количество документов: 999
366
Количество уникальных токенов: 13284
Количество документов: 1165
367
Количество уникальных токенов: 13284
Количество документов: 780
368
Количество уникальных токенов: 13284
Количество документов: 1060
369
Количество уникальных токенов: 13284
Количество документов: 890
370
Количество уникальных токенов: 13284
Количество документов: 898
371
Количество уникальных токенов: 13284
Количество документов: 1138
372
Количество уникальных токенов: 13284
Количество документов: 1266
373
Количество уникальных токенов: 13284
Количество документов: 531
374
Количество уникальных токенов: 13284
Количество документов: 641
375
Количество уникальных токенов: 13284
Количество документов: 1930
376
Количество уникальных токенов: 13284
Количество документов: 1439
377
Количество уникальных токенов: 13284
К

483
Количество уникальных токенов: 13284
Количество документов: 861
484
Количество уникальных токенов: 13284
Количество документов: 400
485
Количество уникальных токенов: 13284
Количество документов: 1559
486
Количество уникальных токенов: 13284
Количество документов: 1047
487
Количество уникальных токенов: 13284
Количество документов: 1140
488
Количество уникальных токенов: 13284
Количество документов: 1577
489
Количество уникальных токенов: 13284
Количество документов: 721
490
Количество уникальных токенов: 13284
Количество документов: 1127
491
Количество уникальных токенов: 13284
Количество документов: 1265
492
Количество уникальных токенов: 13284
Количество документов: 1182
493
Количество уникальных токенов: 13284
Количество документов: 991
494
Количество уникальных токенов: 13284
Количество документов: 1293
495
Количество уникальных токенов: 13284
Количество документов: 438
496
Количество уникальных токенов: 13284
Количество документов: 1437
497
Количество уникальных токенов: 1328

...

2616
Количество уникальных токенов: 13284
Количество документов: 1103
2617
Количество уникальных токенов: 13284
Количество документов: 896
2618
Количество уникальных токенов: 13284
Количество документов: 1524
2619
Количество уникальных токенов: 13284
Количество документов: 1512
2620
Количество уникальных токенов: 13284
Количество документов: 25
2621
Количество уникальных токенов: 13284
Количество документов: 1548
2622
Количество уникальных токенов: 13284
Количество документов: 673
2623
Количество уникальных токенов: 13284
Количество документов: 944
2624
Количество уникальных токенов: 13284
Количество документов: 1641
2625
Количество уникальных токенов: 13284
Количество документов: 955
2626
Количество уникальных токенов: 13284
Количество документов: 956
2627
Количество уникальных токенов: 13284
Количество документов: 943
2628
Количество уникальных токенов: 13284
Количество документов: 699
2629
Количество уникальных токенов: 13284
Количество документов: 534
2630
Количество уникальных ток

2734
Количество уникальных токенов: 13284
Количество документов: 861
2735
Количество уникальных токенов: 13284
Количество документов: 350
2736
Количество уникальных токенов: 13284
Количество документов: 1213
2737
Количество уникальных токенов: 13284
Количество документов: 748
2738
Количество уникальных токенов: 13284
Количество документов: 1357
2739
Количество уникальных токенов: 13284
Количество документов: 450
2740
Количество уникальных токенов: 13284
Количество документов: 461
2741
Количество уникальных токенов: 13284
Количество документов: 1363
2742
Количество уникальных токенов: 13284
Количество документов: 1323
2743
Количество уникальных токенов: 13284
Количество документов: 1732
2744
Количество уникальных токенов: 13284
Количество документов: 1581
2745
Количество уникальных токенов: 13284
Количество документов: 1594
2746
Количество уникальных токенов: 13284
Количество документов: 1044
2747
Количество уникальных токенов: 13284
Количество документов: 434
2748
Количество уникальных

2852
Количество уникальных токенов: 13284
Количество документов: 417
2853
Количество уникальных токенов: 13284
Количество документов: 1124
2854
Количество уникальных токенов: 13284
Количество документов: 2026
2855
Количество уникальных токенов: 13284
Количество документов: 454
2856
Количество уникальных токенов: 13284
Количество документов: 1020
2857
Количество уникальных токенов: 13284
Количество документов: 725
2858
Количество уникальных токенов: 13284
Количество документов: 696
2859
Количество уникальных токенов: 13284
Количество документов: 977
2860
Количество уникальных токенов: 13284
Количество документов: 1439
2861
Количество уникальных токенов: 13284
Количество документов: 633
2862
Количество уникальных токенов: 13284
Количество документов: 1297
2863
Количество уникальных токенов: 13284
Количество документов: 387
2864
Количество уникальных токенов: 13284
Количество документов: 1296
2865
Количество уникальных токенов: 13284
Количество документов: 467
2866
Количество уникальных т

2970
Количество уникальных токенов: 13284
Количество документов: 843
2971
Количество уникальных токенов: 13284
Количество документов: 794
2972
Количество уникальных токенов: 13284
Количество документов: 434
2973
Количество уникальных токенов: 13284
Количество документов: 1502
2974
Количество уникальных токенов: 13284
Количество документов: 1336
2975
Количество уникальных токенов: 13284
Количество документов: 1321
2976
Количество уникальных токенов: 13284
Количество документов: 636
2977
Количество уникальных токенов: 13284
Количество документов: 1154
2978
Количество уникальных токенов: 13284
Количество документов: 1323
2979
Количество уникальных токенов: 13284
Количество документов: 1225
2980
Количество уникальных токенов: 13284
Количество документов: 1214
2981
Количество уникальных токенов: 13284
Количество документов: 1498
2982
Количество уникальных токенов: 13284
Количество документов: 1396
2983
Количество уникальных токенов: 13284
Количество документов: 1298
2984
Количество уникальн

3088
Количество уникальных токенов: 13284
Количество документов: 789
3089
Количество уникальных токенов: 13284
Количество документов: 1174
3090
Количество уникальных токенов: 13284
Количество документов: 1250
3091
Количество уникальных токенов: 13284
Количество документов: 2036
3092
Количество уникальных токенов: 13284
Количество документов: 932
3093
Количество уникальных токенов: 13284
Количество документов: 1144
3094
Количество уникальных токенов: 13284
Количество документов: 1271
3095
Количество уникальных токенов: 13284
Количество документов: 1361
3096
Количество уникальных токенов: 13284
Количество документов: 1761
3097
Количество уникальных токенов: 13284
Количество документов: 880
3098
Количество уникальных токенов: 13284
Количество документов: 413
3099
Количество уникальных токенов: 13284
Количество документов: 1130
3100
Количество уникальных токенов: 13284
Количество документов: 1244
3101
Количество уникальных токенов: 13284
Количество документов: 1226
3102
Количество уникальн

In [31]:
keywords_train = []
for i in range(len(list_topics_train)):
    temp_list = []
    for j in range(len(list_topics_train[i])):
        for k in range(len(list_topics_train[i][j])):
            temp_list.append(list_topics_train[i][j][k][0])
    keywords_train.append(temp_list)

In [32]:
df_train = pd.DataFrame()
df_train['udc'] = y_train
df_train['keywords'] = keywords_train

In [33]:
df_train.to_csv('C:/Users/lalsk/Desktop/d/статьи/keywords_train.csv')

In [51]:
keywords_train_string = []
for i in range(len(df_train['keywords'].tolist())):
    keywords_train_string.append(" ".join(df_train['keywords'].tolist()[i]))

тестовая выборка

In [32]:
text_clean_test= []
for i in range(len(X_test)):
    temp = []
    for j in list(X_test[i].lower().split(' ')):
                temp.append(j.split())
    text_clean_test.append(temp)

In [33]:
bigrams_test = []
trigrams_test= []
for i in range(len(text_clean_test)):
    bigram = Phrases(text_clean_test[i]) # Создаем биграммы на основе корпуса
    trigram = Phrases(bigram[text_clean_test[i]])# Создаем триграммы на основе корпуса
    bigrams_test.append(bigram)
    trigrams_test.append(trigram)

In [34]:
for j in range(len(text_clean_test)):
    for i in range(len(text_clean_test[j])):
        for token in bigrams_test[j][text_clean_test[j][i]]:
            if '_' in token:
                # Токен это биграмма, добавим в документ.
                text_clean_test[j][i].append(token)
        for token in trigrams_test[j][text_clean_test[j][i]]:
            if '_' in token:
                text_clean_test[j][i].append(token)

In [35]:
for i in range(len(text_clean_test)):
    for j in range(len(text_clean_test[i])):
        text_clean_test[i][j].append(' 0')

In [40]:
list_topics_test = []
count = 0
for i in range(len(X_test)):
    count+=1
    print(count)
    corpus = [dictionary.doc2bow(doc) for doc in text_clean_test[i]]
    print('Количество уникальных токенов: %d' % len(dictionary))
    print('Количество документов: %d' % len(corpus))
    model=LdaMulticore(corpus=corpus,id2word=dictionary, num_topics=14)
    temp_list = []
    if (model != 0):
        for j in range(4):
            temp_list.append(model.show_topic(j,topn=14))
    else: 
        temp_list.append('0')
    list_topics_test.append(temp_list)

1


AttributeError: '_io.TextIOWrapper' object has no attribute 'doc2bow'

In [41]:
keywords_test = []
for i in range(len(list_topics_test)):
    temp_list = []
    for j in range(len(list_topics_test[i])):
        for k in range(len(list_topics_test[i][j])):
            temp_list.append(list_topics_test[i][j][k][0])
    keywords_test.append(temp_list)

In [42]:
df_test = pd.DataFrame()
df_test['udc'] = y_test
df_test['keywords'] = keywords_test

In [43]:
df_test.to_csv('C:/Users/lalsk/Desktop/d/статьи/keywords_test.csv')

In [53]:
keywords_test_string = []
for i in range(len(df_test['keywords'].tolist())):
    keywords_test_string.append(" ".join(df_test['keywords'].tolist()[i]))

векторизация

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=1000,max_df=0.7)
keywords_train_vectorized = tfidfconverter.fit_transform(keywords_train_string).toarray()

In [63]:
keywords_test_vectorized = []
for i in range(len(keywords_test_string)):
    temp1 = keywords_train_string.copy()
    temp1.append(keywords_test_string[i])
    temp2 = tfidfconverter.fit_transform(temp1).toarray()
    keywords_test_vectorized.append(temp2[-1])

In [68]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=100, random_state=0)
model = classifier.fit(keywords_train_vectorized, y_train)

In [69]:
y_pred = classifier.predict(keywords_test_vectorized)

In [70]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[  7  17   0   0   0   0   0   0   0   0  28   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  4  76   0   0   0   0   0   0   0   0  49   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   2   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   1   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   1   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   1   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   1   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   2   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   4   0   0   0   0   0   0   0   0  14   0   0   0   0   

C:\Users\lalsk\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lalsk\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lalsk\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [71]:
joblib.dump(model,"Rforest.pkl")

['Rforest.pkl']

In [95]:
report = pd.crosstab(np.array(y_test), np.array(y_pred), rownames=['True'], colnames=['Predicted'], margins=True)
report

Predicted,510,512,513.7,515.1,517,All
True,,,,,,
510,7,17,0,0,28,52
512,4,76,0,0,49,129
513.7,0,0,0,0,2,2
513.74,0,0,0,0,1,1
513.8,0,0,0,0,1,1
513.83,0,0,0,0,1,1
513.88,0,0,0,0,1,1
513.881,0,0,0,0,2,2
514,0,4,0,0,14,18


In [96]:
print(report.to_latex(index=False))

\begin{tabular}{rrrrrr}
\toprule
 510 &  512 &  513.7 &  515.1 &  517 &  All \\
   7 &   17 &      0 &      0 &   28 &   52 \\
\midrule
   4 &   76 &      0 &      0 &   49 &  129 \\
   0 &    0 &      0 &      0 &    2 &    2 \\
   0 &    0 &      0 &      0 &    1 &    1 \\
   0 &    0 &      0 &      0 &    1 &    1 \\
   0 &    0 &      0 &      0 &    1 &    1 \\
   0 &    0 &      0 &      0 &    1 &    1 \\
   0 &    0 &      0 &      0 &    2 &    2 \\
   0 &    4 &      0 &      0 &   14 &   18 \\
   0 &    0 &      0 &      0 &    6 &    6 \\
   1 &   17 &      1 &      2 &  240 &  261 \\
   0 &    0 &      0 &      0 &    1 &    1 \\
   0 &    0 &      0 &      0 &    1 &    1 \\
   0 &    0 &      0 &      0 &    1 &    1 \\
   0 &    3 &      0 &      0 &    1 &    4 \\
   0 &    3 &      0 &      0 &    6 &    9 \\
   0 &    0 &      0 &      0 &    1 &    1 \\
   0 &    1 &      0 &      0 &    0 &    1 \\
   0 &    0 &      0 &      0 &    1 &    1 \\
   0 &    1 &     

In [97]:
from gensim import corpora
corpora.Dictionary.save(dictionary, "dictionary.dict")

In [11]:
df = pd.read_csv('C:/Users/lalsk/Desktop/d/статьи/keywords_test.csv')

In [16]:
df['keywords'][1]

"['функц', 'лиш', 'перв', 'зам', 'уд', 'выполня', 'след', 'имеет', 'х', 'крайн', 'содержат', 'буд', 'о', 'кажд', 'решен', 'х', 'рассмотр', 'существ', 'интеграл', 'отличн', 'област', 'неединствен', 'огранич', 'ф', 'ад', 'ми', 'удк', 'кош', 'некотор', 'л', 'буд', 'класс', 'ввид', 'единствен', 'крив', 'отмет', 'след', 'лебег', 'н', 'услов', 'е', 'име', 'указа', 'случа', 'согласн', 'двух', 'прежд', 'обзор', 'линейн', 'ни', 'неединствен', 'наш', 'обе', 'сдела', '–', 'подынтегральн']"

In [15]:
df

,Unnamed: 0,udc,keywords
0,0,512,"['обознача', 'доказательств', 'чисел', 'множес..."
1,1,517,"['функц', 'лиш', 'перв', 'зам', 'уд', 'выполня..."
2,2,512,"['\x11', '\x17', '\x04', '\x0f\x08', '\x03', '..."
3,3,517,"['лемм', 'сист', 'так', 'след', 'цалюк', 'огра..."
4,4,512,"['неравенств', 'сиб', 'стат', 'поступ', 'ван',..."
...,...,...,...
505,505,514,"['одн', 'произвольн', 'явля', 'знак', 'называ'..."
506,506,517,"['систем', 'е', 'асимптотическ', 'ру', 'свойст..."
507,507,517,"['функц', 'решен', 'задач', 'уравнен', 'рассмо..."
508,508,512,"['факт', 'люб', 'числ', 'эт', 'пересека', 'тив..."
